In [1]:
import pickle

import tensorflow as tf
from pathlib import Path
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import umap
from common.models import AutoEncoder, Encoder, Decoder, MultiHeadAutoEncoder
from common.data_loading import load_all_data
import numpy as np
from sklearn.metrics import confusion_matrix

2022-11-14 14:14:09.630237: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-14 14:14:09.630323: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
X_train, X_val, X_test, y_train, y_val, y_test = load_all_data(1, Path('../data'))

In [3]:
model = MultiHeadAutoEncoder()
model(np.zeros((16, 37)))
model.load_weights('../supervised_detection/day1_420_model.h5')
with open('../supervised_detection/day1_420_spheres.h5', 'rb') as f:
    spheres = pickle.load(f)
with open('../supervised_detection/round-1-scaler.pckl', 'rb') as f:
    scaler = pickle.load(f)

2022-11-14 14:14:23.461824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-14 14:14:23.461882: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-14 14:14:23.461920: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (matterhorn): /proc/driver/nvidia/version does not exist
2022-11-14 14:14:23.462477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
X_test_ = scaler.transform(X_train)
embeded = model.embed(X_test_)

In [6]:
embeded

<tf.Tensor: shape=(5249, 10), dtype=float32, numpy=
array([[4.25865389e-02, 2.26102144e-04, 5.99537373e-01, ...,
        4.31183842e-04, 8.99521708e-01, 2.52022408e-02],
       [3.18743773e-02, 1.23728139e-04, 5.77382445e-01, ...,
        2.88824434e-04, 9.29185450e-01, 3.17197368e-02],
       [2.77825519e-02, 1.03448285e-04, 5.83354414e-01, ...,
        3.05593014e-04, 9.46988583e-01, 3.97177301e-02],
       ...,
       [4.26076017e-02, 2.26355391e-04, 5.99567592e-01, ...,
        4.31274122e-04, 8.99440050e-01, 2.51864698e-02],
       [4.26190868e-02, 2.26393342e-04, 5.99605501e-01, ...,
        4.31013730e-04, 8.99357736e-01, 2.51677558e-02],
       [3.57593186e-02, 1.33852634e-04, 5.87519705e-01, ...,
        2.48289522e-04, 9.13778126e-01, 2.55897418e-02]], dtype=float32)>

In [17]:
y_preds = []
for client_spheres in spheres.values():
    if 1.0 in client_spheres:
        centroid, radius = client_spheres[1.0]
        dist = np.linalg.norm(embeded - centroid, axis=1)
        y_preds.append(dist < radius)

In [18]:
(np.array(y_preds).any(axis=0).astype(np.float32) == y_train).mean()

0.9348447323299676

In [19]:
confusion_matrix(y_train, np.array(y_preds).any(axis=0).astype(np.float32))

array([[4905,  142],
       [ 200,    2]])

In [20]:
spheres

{1: {0.0: (array([ 0.17688444, -0.21995965,  0.23534077, -0.01777917,  0.97534254,
           0.66377403,  0.16445086, -0.10040791,  1.17353539,  0.11898378]),
   0.47605368196964265),
  1.0: (array([ 3.00545011e-02, -5.88514969e-02,  5.20450648e-01, -6.38714314e-02,
           9.10760128e-01,  5.86690503e-01, -8.15185189e-04,  5.04501591e-02,
           8.67433397e-01,  4.50651897e-02]),
   0.14599188268184662)},
 2: {0.0: (array([ 0.21223693, -0.26304423,  0.24435   , -0.02119259,  0.97161306,
           0.72780346,  0.19666324, -0.1202048 ,  1.21712161,  0.12513712]),
   0.5693001419305801),
  1.0: (array([ 1.24858955e-02, -5.47781779e-02,  5.48725124e-01, -6.00185898e-02,
           9.37196907e-01,  3.98141121e-01, -7.59843174e-04,  4.75749187e-02,
           8.64563030e-01,  5.87021076e-02]),
   0.13586377575993538)},
 3: {0.0: (array([ 0.19637828, -0.23213259,  0.32038382, -0.01839613,  0.97018388,
           0.74320647,  0.17357487, -0.10604476,  1.18972816,  0.11830305]),
   0.

In [7]:
embeded
y_preds = []
y_pred_pos = []
for client_spheres in spheres.values():
    if 1.0 in client_spheres:
        centroid, radius = client_spheres[1.0]
        dist = np.linalg.norm(embeded - centroid, axis=1)
        y_preds.append(dist < radius)

    centroid, radius = client_spheres[0.0]
    dist = np.linalg.norm(embeded - centroid, axis=1)
    y_pred_pos.append(dist > radius)

In [41]:
(~(np.array(y_pred_pos).mean(axis=0) > 0.4) == y_train).mean()

0.9615164793293961

In [10]:
_epsilon = 1e-9
def compute_positive_loss(spheres, embedding):
    cls_losses = np.zeros((embedding.shape[0], 2))
    for client in spheres.values():
        for cls, (center, radius) in client.items():
            x = embedding
            safe_norm = np.sqrt(
                np.square(x - center).sum(axis=1) + _epsilon
            )
            dist = safe_norm #tf.norm(x - center, ord='euclidean', axis=1)
            # dist = tf.clip_by_value(dist, 0, tf.float32.max)
            cls_losses[:, int(cls)] += dist #tf.math.reduce_sum(dist, axis=1)
    return cls_losses

def compute_negative_loss(spheres, embedding):
    cls_losses = np.zeros((embedding.shape[0], 2))
    for client in spheres.values():
        for cls, (center, radius) in client.items():
            x_neg = embedding
            safe_norm = np.sqrt(
                np.square(x_neg - center).sum(axis=1) + _epsilon
            )
            dist = radius - safe_norm #tf.norm(x_neg - center, ord='euclidean', axis=1)
            dist = np.clip(dist, 0, tf.float32.max)
            #dist = tf.math.square(dist)
            cls_losses[:, int(cls)] += dist #tf.math.reduce_sum(dist, axis=1)
    return cls_losses
neg_loss = compute_negative_loss(spheres, embeded)
pos_loss = compute_positive_loss(spheres, embeded)

loss = neg_loss + pos_loss


In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, -loss[:, 1])

0.9392794853897113

In [30]:
loss[y_train==0.0, 0].mean()

5.507546514710437